In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from keras import layers, models, losses
from keras.layers import Dense, Dropout, concatenate
import cv2 as cv
import os
import librosa
import moviepy.editor as mp

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  print("gpu")
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5120)])
  except RuntimeError as e:
    print(e)

gpu


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pwd

In [3]:
!pip install tf-models-official

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 81.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 10.9 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=39a1f7138f0adda711bf6f69e091b4b12599ccdfe646b5dcdde5db46a5538a27
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [4]:
import os
os.chdir('/content/drive/MyDrive/Codes/')
!pwd

import importlib

import custom_file_movinet
# Reload the module to ensure the latest version is used
importlib.reload(custom_file_movinet)
from custom_file_movinet import *

/content/drive/MyDrive/Codes


In [5]:
video_model=get_vid_model()

Model: "movinet_classifier_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 image (InputLayer)          [(None, None, None, Non   0         
                             e, 3)]                              
                                                                 
 movinet (Movinet)           ({'stem': (None, None,    911583    
                             None, None, 8),                     
                              'block0_layer0': (None             
                             , None, None, None, 8),             
                              'block1_layer0': (None             
                             , None, None, None, 32)             
                             , 'block1_layer1': (Non             
                             e, None, None, None, 32             
                             ),                                  
                              'block1_layer2':

In [6]:
aud_model=create_vggish_aud_model()


Load trained models

In [7]:
#from tensorflow.keras.applications import ResNet50

video_model.load_weights(r'/content/drive/MyDrive/Codes/Saved models/Saved Movinet/1_without_augmentation/models_new/final_vid_model_MoViNet_1.h5')
aud_model.load_weights(r"/content/drive/MyDrive/Codes/Saved models/Audio_model_multiclass_new.h5")

video_model.summary()
aud_model.summary()



Model: "movinet_classifier_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 image (InputLayer)          [(None, None, None, Non   0         
                             e, 3)]                              
                                                                 
 movinet (Movinet)           ({'stem': (None, None,    911583    
                             None, None, 8),                     
                              'block0_layer0': (None             
                             , None, None, None, 8),             
                              'block1_layer0': (None             
                             , None, None, None, 32)             
                             , 'block1_layer1': (Non             
                             e, None, None, None, 32             
                             ),                                  
                              'block1_layer2':

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 8, 128)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 32)                  │          20,608 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 16)                  │             528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 7)                   │             119 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 21,255 (83.03 KB)

 Trainable params: 21,255 (83.03 KB)

 Non-trainable params: 0 (0.00 B)

### Create audio-video combined function

In [8]:
#@title Helper functions

vggish_model = hub.load('https://tfhub.dev/google/vggish/1')

max_seconds = 8
max_frames = 10

def get_audio_data(path, max_seconds=10):
    audio_data, sample_rate = librosa.load(path, sr=16000)
    aud_len = audio_data.shape[0]
    max_samples = max_seconds * 16000

    if aud_len < max_samples:
        audio_data = np.pad(audio_data, (0, max_samples - aud_len), mode='constant')
    elif aud_len > max_samples:
        audio_data = audio_data[0:max_samples]

    frames = librosa.util.frame(audio_data, frame_length=16000, hop_length=16000)
    frames = np.asarray(frames.T).astype(np.float32)

    features = []
    for frame in frames:
        feature = vggish_model(frame)
        features.append(feature)
    features = tf.concat(features, axis=0)
    return features

def preprocess_video_frames(video_frames):
    processed_frames = []
    for frame in video_frames:
        frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        resized_frame = cv.resize(frame_rgb, (224, 224))/255.0
        processed_frames.append(resized_frame)
    return np.array(processed_frames)

def extract_video_frames(video_path, start_time, end_time, num_frames=max_frames):
    cap = cv.VideoCapture(video_path)
    fps = cap.get(cv.CAP_PROP_FPS)
    total_frames = int((end_time - start_time) * fps)
    frame_interval = total_frames // num_frames

    frames = []
    for i in range(num_frames):
        frame_number = start_time * fps + i * frame_interval
        cap.set(cv.CAP_PROP_POS_FRAMES, frame_number)
        ret, frame = cap.read()
        if ret:
            frames.append(frame)
        else:
            break
    cap.release()
    return frames

def run_classification_on_clip(video_path, start_time, end_time):

    classes_list = ['Abuse', 'Riot', 'Car Accident', 'Nonviolence', 'Shooting', 'Explosion', 'Fighting']
    # Extract video frames
    video_frames = extract_video_frames(video_path, start_time, end_time)
    processed_video_frames = preprocess_video_frames(video_frames)

    # Extract audio features
    clip = mp.VideoFileClip(video_path).subclip(start_time, end_time)
    audio_path = 'temp_audio.wav'
    clip.audio.write_audiofile(audio_path, codec='pcm_s16le')
    audio_features = get_audio_data(audio_path,max_seconds)

    # Run predictions
    video_predictions = video_model.predict(np.expand_dims(processed_video_frames, axis=0))
    video_predictions = tf.keras.layers.Softmax()(video_predictions)
    audio_predictions = aud_model.predict(np.expand_dims(audio_features, axis=0))

    # Combine predictions
    print(f"video_predictions {video_predictions}")
    print(f"audio_predictions {audio_predictions}")
    print(f"video_predicted class {classes_list[np.argmax(video_predictions)]}")
    print(f"audio_predicted class {classes_list[np.argmax(audio_predictions)]}")
    combined_predictions = (video_predictions + audio_predictions) / 2
    final_class = np.argmax(combined_predictions)
    final_class = classes_list[final_class]

    # Clean up temporary audio file
    os.remove(audio_path)

    return final_class

# Process the entire movie in 10-second clips
def process_movie(video_path):
    classes_list = ['Abuse', 'Riot', 'Car Accident', 'Nonviolence', 'Shooting', 'Explosion', 'Fighting']
    # Get the total duration of the video
    cap = cv.VideoCapture(video_path)
    total_duration = int(cap.get(cv.CAP_PROP_FRAME_COUNT) / cap.get(cv.CAP_PROP_FPS))
    cap.release()

    clip_duration = max_seconds#10
    class_counts = {}
    total_clips = 0

    for start_time in range(0, total_duration, clip_duration):
        end_time = min(start_time + clip_duration, total_duration)
        if end_time - start_time < clip_duration:
            continue#########################
        final_class = run_classification_on_clip(video_path, start_time, end_time)
        class_counts[final_class] = class_counts.get(final_class, 0) + 1
        total_clips += 1
        print(f"Clip {start_time}-{end_time} classified as class {final_class}")

    print("\nSummary of the whole video:")
    for cls, count in class_counts.items():
        percentage = (count / total_clips) * 100
        print(f"Class {cls}: {percentage:.2f}%")




In [20]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

csv_path= "/content/drive/MyDrive/Codes/CSVs/1_First_method_vid_enhanced_for_multiple_classes"

if 'csv_path' not in vars():
    csv_path ="/content/drive/MyDrive/Codes/CSVs/1_First_method_vid_enhanced_for_multiple_classes"

#loaded_train_df = pd.read_csv(os.path.join(csv_path,"train.csv"),usecols=['Name','tag'])

loaded_test_df = pd.read_csv(os.path.join(csv_path,"test.csv"),usecols=['Name','tag'])

loaded_test_df.head()

def run_classification_on_clip_for_eval(video_path, start_time, end_time):

    classes_list = ['Abuse', 'Riot', 'Car Accident', 'Nonviolence', 'Shooting', 'Explosion', 'Fighting']
    # Extract video frames
    video_frames = extract_video_frames(video_path, start_time, end_time)
    processed_video_frames = preprocess_video_frames(video_frames)

    # Extract audio features
    clip = mp.VideoFileClip(video_path).subclip(start_time, end_time)
    audio_path = 'temp_audio.wav'
    clip.audio.write_audiofile(audio_path, codec='pcm_s16le')
    audio_features = get_audio_data(audio_path,max_seconds)

    # Run predictions
    video_predictions = video_model.predict(np.expand_dims(processed_video_frames, axis=0))
    video_predictions = tf.keras.layers.Softmax()(video_predictions)
    audio_predictions = aud_model.predict(np.expand_dims(audio_features, axis=0))

    # Combine predictions
    combined_predictions = (video_predictions + audio_predictions) / 2
    final_class = np.argmax(combined_predictions)
    final_class = classes_list[final_class]

    # Clean up temporary audio file
    os.remove(audio_path)

    return final_class

# Process the entire movie in 10-second clips
def pred_vid(video_path):
    classes_list = ['Abuse', 'Riot', 'Car Accident', 'Nonviolence', 'Shooting', 'Explosion', 'Fighting']
    # Get the total duration of the video
    cap = cv.VideoCapture(video_path)
    total_duration = int(cap.get(cv.CAP_PROP_FRAME_COUNT) / cap.get(cv.CAP_PROP_FPS))
    cap.release()

    clip_duration = max_seconds#10  # since our clips are short this will only capture the clip
    class_counts = {}
    total_clips = 0

    for start_time in range(0, total_duration, clip_duration):
        end_time = min(start_time + clip_duration, total_duration)
        final_class = run_classification_on_clip_for_eval(video_path, start_time, end_time)
        return final_class  # This will only work for clips around 10 seconds

def vid_eval(df):
    classes_list = ['Abuse', 'Riot', 'Car Accident', 'Nonviolence', 'Shooting', 'Explosion', 'Fighting']
    df= pd.get_dummies(df.loc[:,['Name','tag']], columns=['tag'])
    df.iloc[:,1:]= df.iloc[:,1:].astype(np.uint8)
    confusion_matrix = np.zeros((7,7))
    for i in range(len(loaded_test_df)):
      path = loaded_test_df.iloc[i,0]
      label = loaded_test_df.iloc[i,1]

      try:
        pred_class = pred_vid(path)
      except Exception as exc:
        print(exc)
        continue

      print(f"label-{label} \n pred- {pred_class}")
      confusion_matrix[classes_list.index(label),classes_list.index(pred_class)]+=1
    accuracy = np.trace(confusion_matrix)/np.sum(confusion_matrix)
    print(f"accuracy {accuracy}")
    ax = sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', xticklabels=classes_list, yticklabels=classes_list)
    ax.set_xlabel('Predicted labels')
    ax.set_ylabel('True labels')
    ax.set_title('Confusion Matrix')
    plt.show()
    return confusion_matrix


In [21]:
cm=vid_eval(loaded_test_df.sample(frac = 1))

MoviePy - Writing audio in temp_audio.wav


MoviePy - Done.


1/1 [==============================] - 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
label-Abuse 
 pred- Abuse
MoviePy - Writing audio in temp_audio.wav


MoviePy - Done.


1/1 [==============================] - 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
label-Abuse 
 pred- Nonviolence
MoviePy - Writing audio in temp_audio.wav


...

MoviePy - Done.


1/1 [==============================] - 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
label-Fighting 
 pred- Abuse
MoviePy - Writing audio in temp_audio.wav


MoviePy - Done.


1/1 [==============================] - 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
label-Fighting 
 pred- Nonviolence
accuracy 0.13793103448275862


array([[37.,  0.,  0.,  8.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0., 22.,  0.],
       [ 1., 27.,  9.,  2.,  2.,  1.,  0.],
       [11., 22.,  3.,  7., 49.,  2.,  3.],
       [13.,  8.,  7., 20.,  2.,  2., 30.],
       [ 1.,  2., 25.,  3.,  1.,  1.,  2.],
       [ 9.,  0.,  2., 88.,  3.,  5.,  4.]])

In [38]:
# Example usage
video_path = r'/content/drive/MyDrive/Multi Class Dataset/Fighting/1065.mp4'
process_movie(video_path)

MoviePy - Writing audio in temp_audio.wav


MoviePy - Done.


1/1 [==============================] - 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
video_predictions [[0.12020984 0.0053605  0.01900598 0.20392457 0.1134651  0.26439118
  0.27364287]]
audio_predictions [[0.06488045 0.00085516 0.00394835 0.7902981  0.03679961 0.00399699
  0.0992214 ]]
video_predicted class Fighting
audio_predicted class Nonviolence
Clip 0-8 classified as class Nonviolence

Summary of the whole video:
Class Nonviolence: 100.00%
